In [1]:
# Please visit:
#
# https://download.nkg-mn.com/credo/anomalies/
#
# Pretrained dots_100000.h5 and others can be downloaded from mentioned website.

In [2]:
from pyimagesearch.vae import build_vae_autoencoder, train_vae
from commons import *
from dataset_loader import load_from_file
from IPython.display import display
import matplotlib.pyplot as plt
import cv2

DOTS_DST = 'cache/dots_100000.pickle'
TRACKS_DST = 'cache/tracks_100000.pickle'
WORMS_DST = 'cache/worms_100000.pickle'
ARTIFACTS_DST = 'cache/artifacts_100000.pickle'

DOTS_AC = 'cache/dots_vae_100000.tf'
TRACKS_AC = 'cache/tracks_vae_100000.tf'
WORMS_AC = 'cache/worms_vae_100000.tf'
ARTIFACTS_AC = 'cache/artifacts_vae_100000.tf'


def train_and_test(train_set, validation_set, fn):
    _, __, autoencoder = build_vae_autoencoder()
    autoencoder = train_vae(autoencoder, fn, train_set=train_set, validation_set=validation_set)

    for df_func, df_name in zip(
            [dm_func_mean],
            ['mean']
    ):
        for img_set, set_names in zip([validation_set], ['train']):
            decoded = autoencoder.predict(img_set)

            vis, errors = visualize_predictions(decoded, img_set, df_func, False, 16)
            img_path = fn.replace('.h5', '-vis.png')
            hist_path = fn.replace('.h5', '-hist.png')
            cv2.imwrite(img_path, vis)
            display(Image.open(img_path))
            if len(img_set) > 256:
                vis, errors = visualize_predictions(decoded, img_set, df_func, False)
                img_path = fn.replace('.h5', '-vis_full.png')
                cv2.imwrite(img_path, vis)

            plt.figure(figsize=(8,6))
            plt.hist(errors, bins=100, alpha=0.5, label=set_names)
            plt.xlabel("Data", size=14)
            plt.ylabel("Count", size=14)
            plt.title(df_name)
            plt.legend(loc='upper right')
            plt.savefig(hist_path)


def load_and_train(src, out):
    images, augmented = load_from_file(src)
    train_and_test(np.expand_dims(augmented, axis=-1), np.expand_dims(images, axis=-1), out)

In [3]:
load_and_train(DOTS_DST, DOTS_AC)

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 14, 14, 32)   320         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 7, 7, 64)     18496       ['conv2d[0][0]']                 
                                                                                                  
 flatten (Flatten)              (None, 3136)         0           ['conv2d_1[0][0]']               
                                                                                            

ValueError: Specified input shape is not one of the valid types. Please specify a batch input shape of type tuple or list of input shapes. User provided input type: <class 'numpy.ndarray'>.

In [ ]:
load_and_train(TRACKS_DST, TRACKS_AC)

In [ ]:
load_and_train(WORMS_DST, WORMS_AC)

In [ ]:
load_and_train(ARTIFACTS_DST, ARTIFACTS_AC)